In [ ]:
# import libraries
from dataloader import get_dataloader_vae
from dataloader import get_dataloader_OOD
from models import get_trained_model
from energy import ELOOD

# Get model, dataloaders, and OOD detectors

In [ ]:
# get model
model = get_trained_model('CIFAR10')

# get dataloaders
train_dl_c10, _ = get_dataloader_vae('CIFAR10')
train_dl_c100, _ = get_dataloader_vae('CIFAR100')
svhn_dl = get_dataloader_OOD('SVHN')
lsun_dl = get_dataloader_OOD('LSUN')

# get OOD detector
c10_ood = ELOOD(model)
c100_ood = ELOOD(model)

# Train OOD detector for CIFAR 10

In [ ]:
c10_ood.train_ood(train_dl_c10)

# Get predictions for SVHN and LSUN

In [ ]:
svhn_c10_ood = c10_ood.predict_ood(svhn_dl)
lsun_c10_ood = c10_ood.predict_ood(lsun_dl)

# Train OOD detector for CIFAR 100

In [ ]:
c100_ood.train_ood(train_dl_c100)

# Get predictions for SVHN and LSUN

In [ ]:
svhn_c100_ood = c100_ood.predict_ood(svhn_dl)
lsun_c100_ood = c100_ood.predict_ood(lsun_dl)